In [ ]:
from google.cloud import aiplatform

In [ ]:
project_id = "udemy-mlops"
region = "us-central1"
staging_bucket="gs://sid-vertex-mlops"

aiplatform.init(project=project_id, location=region, staging_bucket=staging_bucket)

job = aiplatform.CustomTrainingJob(
        display_name="bikeshare-training-job",
        script_path="model-training-code.py",
        container_uri="us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest",
        requirements=["gcsfs"]
    )

job.run(
    replica_count=1,
    machine_type="n1-standard-4",
    sync=True
)
job.wait()

In [ ]:
display_name = "bikeshare-model-sdk"
artifact_uri = "gs://sid-ml-ops/bike-share-rf-regression-artifact/"
serving_container_image_uri = "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest"

model = aiplatform.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
        sync=False
    )

In [ ]:
deployed_model_display_name = "bikeshare-model-endpoint"
traffic_split = {"0": 100}
machine_type = "n1-standard-4"
min_replica_count = 1
max_replica_count = 1

endpoint = model.deploy(
        deployed_model_display_name=deployed_model_display_name,
        traffic_split=traffic_split,
        machine_type=machine_type,
        min_replica_count=min_replica_count,
        max_replica_count=max_replica_count
    )

In [ ]:
INSTANCE = [0.24,
 0.81,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0]
instances_list = [INSTANCE]
prediction = endpoint.predict(instances_list)
print(prediction)

In [ ]:
gcs_input_uri = 'gs://sid-vertex-mlops/bike-share/batch-new.csv'
BUCKET_URI = "gs://sid-vertex-mlops/bikeshare-batch-prediction-output"

batch_predict_job = model.batch_predict(
    job_display_name="bikeshare_batch_predict",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="csv",
    predictions_format="jsonl",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=1,
    generate_explanation=True,
    sync=False
)